In [ ]:
import torch
from models import (
    CnnSimpleAgent,
    # DinoSimpleAgent,
    CnnCompoNetAgent,
    CnnTvNetAgent, 
    ProgressiveNetAgent,
    PackNetAgent,
    FuseNetAgent,
    CnnMaskAgent,
    FuseNetwMergeAgent,
    RewireAgent,
    CnnCbpAgent,
    CReLUsAgent,
)

In [ ]:
# load the model
device = "cuda" if torch.cuda.is_available() else "cpu"

if algorithm in ["F1", "FN", "Baseline", "Finetune"]:
    agent = CnnSimpleAgent.load(
        args.load, envs, load_critic=False, map_location=device
    )
elif algorithm == "CompoNet":
    prevs_paths = [path_from_other_mode(args.load, i) for i in range(args.train_mode)]
    agent = CnnCompoNetAgent.load(
        args.load, envs, prevs_paths=prevs_paths, map_location=device
    )
elif algorithm == "ProgNet":
    prevs_paths = [path_from_other_mode(args.load, i) for i in range(args.train_mode)]
    agent = ProgressiveNetAgent.load(
        args.load, envs, prevs_paths=prevs_paths, map_location=device
    )
elif algorithm == "PackNet":
    task_id = None if args.mode == None else args.mode + 1
    agent = PackNetAgent.load(args.load, task_id=task_id, map_location=device)
    agent.network.set_view(task_id)

    if mode != train_mode:
        # load the actor and critic heads from the model trained in the testing task (game mode)
        path = path_from_other_mode(args.load, mode)
        ac = PackNetAgent.load(path, map_location=device)
        agent.critic = ac.critic
        agent.actor = ac.actor
elif algorithm == "FuseNet":
    agent = FuseNetAgent.load(args.load, envs, map_location=device)
elif algorithm == "FuseNetwMerge":
    agent = FuseNetwMergeAgent.load(args.load, envs, map_location=device)
elif algorithm == "MaskNet":
    agent = CnnMaskAgent.load(args.load, envs, num_tasks=get_num_task(env_name), map_location=device)
elif algorithm == "Rewire":
    agent = RewireAgent.load(args.load, envs, map_location=device)
    agent.set_task(args.mode)
elif algorithm == "CReLUs":
    agent = CReLUsAgent.load(args.load, envs, map_location=device)
elif algorithm == "CbpNet":
    agent = CnnCbpAgent.load(args.load, envs, map_location=device)
else:
    print(f"Loading of agent type `{algorithm}` is not implemented.")
    quit(1)

agent.to(device)
